In [1]:
import itertools
from parlay_system import *
pd.set_option('display.max_rows', 500)


In [2]:
# December 16 games
# line 1.5
kentst = MoneyLine(event="kentst", bet_amount=100, odds=104)
uc_irvine = MoneyLine(event="uc_irvine", bet_amount=100, odds=-125)

# line 2
marshall = MoneyLine(event="marshall", bet_amount=100, odds=110)
morehead = MoneyLine(event="morehead", bet_amount=100, odds=-136)

# line 3.5
valpraiso = MoneyLine(event="valpraiso", bet_amount=100, odds=130)
charlotte = MoneyLine(event="charlotte", bet_amount=100, odds=-159)

# line 7.5
montantast = MoneyLine(event="montantast", bet_amount=100, odds=280)
ndakotast = MoneyLine(event="ndakotast", bet_amount=100, odds=-360)

# line 6.5
idahost = MoneyLine(event="idahost", bet_amount=100, odds=235)
ucsantab = MoneyLine(event="ucsantab", bet_amount=100, odds=-295)

binaries = [
#     [kentst, uc_irvine],
    [marshall, morehead],
    [valpraiso, charlotte],
    [montantast, ndakotast],
#     [idahost, ucsantab]
]

results = [
#     [0, 1], #[kentst, uc_irvine],
#     [1, 1], #[marshall, morehead],
    [0, 1], #[valpraiso, charlotte],
    [0, 1], #[montantast, ndakotast],
    [0, 1], #[idahost, ucsantab]
]


def create_dicts(in_binaries, in_results):
    ml_lookup = {}
    ml_win_lookup = {}
    id = 0
    for bi in range(len(in_binaries)):
        binary = in_binaries[bi]
        for ml_i in range(len(binary)):
            ml = binary[ml_i]
            ml.set_index(id) 
            is_winner = in_results[bi][ml_i]
            ml_lookup[id] = ml.event
            ml_win_lookup[id] = is_winner
            id += 1
    
    return ml_lookup, ml_win_lookup

ml_lookup, ml_outcome_lookup = create_dicts(binaries, results)



In [3]:
#ml_lookup

In [4]:
#ml_outcome_lookup

In [5]:
def findsubsets(s, n):
    return list(itertools.combinations(s, n))

TOURNEY_SIZE = len(binaries)
SELECT_NUM = 3
s = set()
n = SELECT_NUM

for i in range(TOURNEY_SIZE):
    s.add(i)

all_subsets = findsubsets(s, n)

In [6]:
len(all_subsets)

1

In [7]:
csv = []
df = pd.DataFrame()

for i in range(len(all_subsets)):
    current_subset = all_subsets[i]
    parlay_binaries = []
    for i in range(len(current_subset)):
        index = current_subset[i]
        parlay_binaries.append(binaries[index])

    ps = ParlaySystem(binaries=parlay_binaries,
                    target_profit=1,
                    bounds=(0.01, 30),
                    binary_index_arr=list(current_subset),
                    binary_results_arr=results,
                    index_to_ml=ml_lookup,
                    index_to_outcome=ml_outcome_lookup
                    )
    csv_res, raw_df = ps.slsqp_solver()
    df = pd.concat([df, raw_df])
    csv.append(csv_res)

     fun: -0.39442615208245263
     jac: array([-1.41925   ,  0.10354167, -0.74985849,  0.32862946, -1.02080882,
        0.23752042, -0.46767018,  0.42351734])
 message: 'Positive directional derivative for linesearch'
    nfev: 70
     nit: 9
    njev: 5
  status: 8
 success: False
       x: array([0.1022758 , 0.14757249, 0.14441026, 0.06760373, 0.12377105,
       0.10677575, 0.1747609 , 0.01      ])
slsqp_solver: 
                           event    index[]     result  event_status   bet  \
7   morehead_charlotte_ndakotast  [1, 3, 5]  [1, 1, 1]          True  0.01   
0  marshall_valpraiso_montantast  [0, 2, 4]  [0, 0, 0]         False  0.10   
2  marshall_charlotte_montantast  [0, 3, 4]  [0, 1, 0]         False  0.14   
4  morehead_valpraiso_montantast  [1, 2, 4]  [1, 0, 0]         False  0.12   
6  morehead_charlotte_montantast  [1, 3, 4]  [1, 1, 0]         False  0.17   
1   marshall_valpraiso_ndakotast  [0, 2, 5]  [0, 0, 1]         False  0.15   
5   morehead_valpraiso_ndakotast  

In [8]:
df.describe()

,bet,mult,payout,profit
count,8.000000,8.000000,8.000000,8.000000
mean,0.108750,9.564379,1.161963,0.285000
std,0.050551,5.550657,0.802578,0.801962
min,0.010000,3.611861,0.036100,-0.840000
25%,0.092500,4.917619,0.482250,-0.392500
50%,0.115000,8.456514,1.394000,0.515000
75%,0.142500,13.540769,1.877200,1.000000
max,0.170000,18.354000,1.877200,1.000000


In [9]:
negative_df = df.loc[df.profit < 0]
negative_df = negative_df.sort_values(by=['event_status', 'profit'], ascending=[False, True])
negative_df.describe()

,bet,mult,payout,profit
count,3.000000,3.000000,3.000000,3.000000
mean,0.063333,4.360887,0.292033,-0.583333
std,0.050332,0.744039,0.254218,0.255016
min,0.010000,3.611861,0.036100,-0.840000
25%,0.040000,3.991413,0.165800,-0.710000
50%,0.070000,4.370964,0.295500,-0.580000
75%,0.090000,4.735400,0.420000,-0.455000
max,0.110000,5.099837,0.544500,-0.330000


In [10]:
positive_df = df.loc[df.profit >= 0]
positive_df = positive_df.sort_values(by=['event_status', 'profit'], ascending=[False, False])
positive_df.describe()

,bet,mult,payout,profit
count,5.000000,5.000000,5.000000,5.000000
mean,0.136000,12.686473,1.683920,0.806000
std,0.027019,4.598777,0.432187,0.433797
min,0.100000,6.171667,0.910800,0.030000
25%,0.120000,10.741361,1.877200,1.000000
50%,0.140000,12.998868,1.877200,1.000000
75%,0.150000,15.166471,1.877200,1.000000
max,0.170000,18.354000,1.877200,1.000000


In [11]:
negative_df.head(200)

,event,index[],result,event_status,bet,mult,payout,profit
7,morehead_charlotte_ndakotast,"[1, 3, 5]","[1, 1, 1]",True,0.01,3.611861,0.0361,-0.84
3,marshall_charlotte_ndakotast,"[0, 3, 5]","[0, 1, 1]",False,0.07,4.370964,0.2955,-0.58
5,morehead_valpraiso_ndakotast,"[1, 2, 5]","[1, 0, 1]",False,0.11,5.099837,0.5445,-0.33


In [12]:
positive_df.head(200)

,event,index[],result,event_status,bet,mult,payout,profit
0,marshall_valpraiso_montantast,"[0, 2, 4]","[0, 0, 0]",False,0.10,18.354000,1.8772,1.00
2,marshall_charlotte_montantast,"[0, 3, 4]","[0, 1, 0]",False,0.14,12.998868,1.8772,1.00
4,morehead_valpraiso_montantast,"[1, 2, 4]","[1, 0, 0]",False,0.12,15.166471,1.8772,1.00
6,morehead_charlotte_montantast,"[1, 3, 4]","[1, 1, 0]",False,0.17,10.741361,1.8772,1.00
1,marshall_valpraiso_ndakotast,"[0, 2, 5]","[0, 0, 1]",False,0.15,6.171667,0.9108,0.03


In [13]:
positive_df.groupby('event_status').count()

,event,index[],result,bet,mult,payout,profit
event_status,,,,,,,
False,5,5,5,5,5,5,5


In [14]:
negative_df.groupby('event_status').count()

,event,index[],result,bet,mult,payout,profit
event_status,,,,,,,
False,2,2,2,2,2,2,2
True,1,1,1,1,1,1,1


In [15]:
#csv

In [16]:
def color_negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    if type(val) != bool:
        return 'color: black'
    
    color = 'red' if val == True else 'black'
    return 'color: %s' % color

def color_positive_green(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    if type(val) != bool:
        return 'color: black'
    
    color = 'green' if val == True else 'black'
    return 'color: %s' % color

In [17]:
neg = negative_df.reset_index(drop=True).style.applymap(color_negative_red)
neg

,event,index[],result,event_status,bet,mult,payout,profit
0,morehead_charlotte_ndakotast,"['1', '3', '5']","[1, 1, 1]",True,0.01,3.61186,0.0361,-0.84
1,marshall_charlotte_ndakotast,"['0', '3', '5']","[0, 1, 1]",False,0.07,4.37096,0.2955,-0.58
2,morehead_valpraiso_ndakotast,"['1', '2', '5']","[1, 0, 1]",False,0.11,5.09984,0.5445,-0.33


In [18]:
pos = positive_df.reset_index(drop=True).style.applymap(color_positive_green)
pos

,event,index[],result,event_status,bet,mult,payout,profit
0,marshall_valpraiso_montantast,"['0', '2', '4']","[0, 0, 0]",False,0.1,18.354,1.8772,1
1,marshall_charlotte_montantast,"['0', '3', '4']","[0, 1, 0]",False,0.14,12.9989,1.8772,1
2,morehead_valpraiso_montantast,"['1', '2', '4']","[1, 0, 0]",False,0.12,15.1665,1.8772,1
3,morehead_charlotte_montantast,"['1', '3', '4']","[1, 1, 0]",False,0.17,10.7414,1.8772,1
4,marshall_valpraiso_ndakotast,"['0', '2', '5']","[0, 0, 1]",False,0.15,6.17167,0.9108,0.03
